---

# 쇼핑몰 로그 데이타 분석하기



#### 아래의 내용은 로그 데이터분석의 핵심 수치들을 구하는 것과 이를 위한 전처리로 이루어져 있습니다. 핵심 수치들은 다음과 같습니다.

1. page duration
    - 사용자가 앱의 한 page당 체류하는 시간입니다. 
<br/>
2. session
    - 앱을 실행하는 단위로서, 세션은 사용자가 앱을 실행한 후부터 그 실행을 마칠 때까지의 일련의 과정을 포함합니다.
<br/>
3. 체류 시간
    - 사용자가 page, session 혹은 특정 기준동안 머무르는 시간입니다.<br/>

[assignment]에서는 각 용어를 "log duration", "cycle", "잔존 시간"이란 표현을 사용하였지만, 앞으로는 로그 분석에서 보편적으로 사용하는 용어인 위의 용어들로 사용하겠습니다. 다음주 분석인 kmong 데이터 분석에서도 같은 용어를 사용할 것입니다. 


[ 참고 ] [groupby 함수 공부](https://pandas.pydata.org/pandas-docs/stable/groupby.html)

## set options

In [1]:
# 데이터를 다루는 library인 padas를 import합니다.
import pandas as pd

# 화면에 출력하는 데이터 프레임의 최대 row 수를 500으로 설정합니다.
pd.set_option('display.max_rows', 500)

# 화면에 출력하는 데이터 프레임의 최대 column 수를 500으로 설정합니다.
pd.set_option('display.max_columns', 500)

**0.  사용자 데이타와 주문 데이타 불러오기**

In [2]:
user = pd.read_csv('./data/shoppingmall/user_info.csv')
user.head()

user_id   os  age
0  --PYPMX8QWg0ioT5zfORmU-S5Lln0lot  And   41
1  -16-xXbeDcvkZJtTpRwMi57Yo2ZQpORv  iOS   31
2  -1de9sT-MLwVVvnC0ncCLnqEqpSi3XSN  iOS   16
3  -3A3L2jnM55B_Q1bRXMjZ6sPnINIj-Y1  And   41
4  -3bhcSgPOIdQAPkPNcchxvECGqGQQ78k  And   42

In [3]:
order = pd.read_csv('./data/shoppingmall/order_info.csv')
order.head()

timestamp                           user_id  goods_id  \
0  2018-06-11 00:00:43.032  bvu0aLTqiFDoU-963xnr5nzQWTNLUMjx      1414   
1  2018-06-11 00:02:33.763  smDmRnykg61KajpxXKzQ0oNkrh2nuSBj      1351   
2  2018-06-11 00:04:06.364  EyGjKYtSqZgqJ1ddKCtH5XwGirTyOH2P       646   
3  2018-06-11 00:04:17.258  KQBGi33Zxh5Dgu0WEkOkjN0YqTT_wxC3      5901   
4  2018-06-11 00:05:26.010  lq1Je3voA3a0MouSFba3629lKCvweI24      5572   

   shop_id  price  
0       38  45000  
1       12   9500  
2       14  22000  
3       46  29800  
4       89  29000

**1. 로그 데이터를 불러와주신 다음 timestamp컬럼을 datetime 형식으로 변환해 주세요. 그리고 user id를 보기 쉽게 간단한 자연수 형태로 변환해주세요.**

In [4]:
data_logs = pd.read_csv('./data/shoppingmall/user_event_logs.csv')
print(data_logs.shape)
data_logs.head()


(105815, 6)


timestamp                           user_id  \
0  2018-06-11 00:00:00.213  K1d8_t3-QIskaSkrx32oAFu856D8JmLo   
1  2018-06-11 00:00:00.810  lwFZ77v_ygk0uU40t1ud3l30EZ6sE2R3   
2  2018-06-11 00:00:00.956  mR-bO6hC9g-m8ERXMRQZaRwJFvzNNdd8   
3  2018-06-11 00:00:01.084  K1d8_t3-QIskaSkrx32oAFu856D8JmLo   
4  2018-06-11 00:00:01.561  Yjny5AchUWLiuv4kdeq50COF-S8OFXPd   

             event_origin     event_name  event_goods_id  event_shop_id  
0           shops_ranking  app_page_view             NaN            NaN  
1          shops_bookmark  app_page_view             NaN            NaN  
2  goods_search_result/로브  app_page_view             NaN            NaN  
3          shops_bookmark  app_page_view             NaN            NaN  
4          shops_bookmark  app_page_view             NaN            NaN

In [5]:
# timestamp 컬럼를 datetime 타입으로 변환해주세요.
data_logs['timestamp'] = pd.to_datetime(data_logs['timestamp'])

order['timestamp'] = pd.to_datetime(order['timestamp'])

지그재그 로그 데이터의 명세는 다음과 같습니다.


- 컬럼 별 명세
        1. timestamp : 이벤트 발생 시간 (한국 시간 기준)
        2. user_id : 이용자 고유 식별자
        3. event_origin : 이벤트가 발생한 앱 위치
            - event_origin 값 별 의미
                a. goods_search_result : 특정 검색어의 상품 검색 결과
                    (Ex: goods_search_result/반팔티)
                b. shops_ranking : '쇼핑몰 랭킹' 영역
                c. shops_bookmark : '즐겨찾기' 영역
                d. category_search_result : 카테고리 검색 결과 
                    (Ex:category_search_result/상의)
                e. my_goods : '내 상품' 영역
                
        4. event_name : 발생한 이벤트 명
            - event_name 값 별 의미
                a. app_page_view : 앱 내 화면 이동
                b. enter_browser : 앱 내 클릭을 통해, 특정 웹페이지로 진입
                c. add_bookmark : 특정 쇼핑몰을 즐겨찾기 추가
                d. remove_bookmark : 특정 쇼핑몰을 즐겨찾기 제거
                e. add_my_goods : 특정 상품을 내 상품 추가
                f. remove_my_goods : 특정 상품을 내 상품 제거
                
        5. event_goods_id : 이벤트가 발생한 상품 고유 식별자
             - 상품 관련 이벤트가 아닌 경우, 공백
             
        6. event_shop_id : 이벤트가 발생한 쇼핑몰 고유 식별자
             - 쇼핑몰 관련 이벤트가 아닌 경우, 공백




##  보안처리된 복잡한 아이디 간소화하기
user_id는 아래에서 확인할 수 있듯이 매우 복잡한 형태로 되어 있어 한눈에 파악하기 어렵습니다.


<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>user_id</th>
      <th>os</th>
      <th>age</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>--PYPMX8QWg0ioT5zfORmU-S5Lln0lot</td>
      <td>And</td>
      <td>41</td>
    </tr>
    <tr>
      <th>1</th>
      <td>-16-xXbeDcvkZJtTpRwMi57Yo2ZQpORv</td>
      <td>iOS</td>
      <td>31</td>
    </tr>
    <tr>
      <th>2</th>
      <td>-1de9sT-MLwVVvnC0ncCLnqEqpSi3XSN</td>
      <td>iOS</td>
      <td>16</td>
    </tr>
    <tr>
      <th>3</th>
      <td>-3A3L2jnM55B_Q1bRXMjZ6sPnINIj-Y1</td>
      <td>And</td>
      <td>41</td>
    </tr>
    <tr>
      <th>4</th>
      <td>-3bhcSgPOIdQAPkPNcchxvECGqGQQ78k</td>
      <td>And</td>
      <td>42</td>
    </tr>
  </tbody>
</table>




앞으로의 분석을 용이하게 하기위하여 user_id을 간단하게 0, 1, 2, 3 ...과 같이 연속된 정수 형태로 변환하여 아래와 같이 만들겠습니다.


<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>user_id</th>
      <th>n_user_id</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>K1d8_t3-QIskaSkrx32oAFu856D8JmLo</td>
      <td>3314</td>
    </tr>
    <tr>
      <th>1</th>
      <td>lwFZ77v_ygk0uU40t1ud3l30EZ6sE2R3</td>
      <td>7844</td>
    </tr>
    <tr>
      <th>2</th>
      <td>mR-bO6hC9g-m8ERXMRQZaRwJFvzNNdd8</td>
      <td>7920</td>
    </tr>
    <tr>
      <th>3</th>
      <td>K1d8_t3-QIskaSkrx32oAFu856D8JmLo</td>
      <td>3314</td>
    </tr>
    <tr>
      <th>4</th>
      <td>Yjny5AchUWLiuv4kdeq50COF-S8OFXPd</td>
      <td>5608</td>
    </tr>
    <tr>
      <th>5</th>
      <td>LZZ0ktGq6hW685TFAQfcGNhsKVUEceHl</td>
      <td>3548</td>
    </tr>
    <tr>
      <th>6</th>
      <td>TUoAGIbbNds5cYLZLnz-R5VlkG5L8RuZ</td>
      <td>4790</td>
    </tr>
    <tr>
      <th>7</th>
      <td>B9F_BHH9F3b6MW329go9jDr71Uunx629</td>
      <td>1902</td>
    </tr>
    <tr>
      <th>8</th>
      <td>e_xrTZ9fHVodxxadLx688qUKMWCdL8bW</td>
      <td>6663</td>
    </tr>
    <tr>
      <th>9</th>
      <td>aA9S7LxnFm6ym6IUEa-4SSxJa-iL5m2J</td>
      <td>5976</td>
    </tr>
  </tbody>
</table>

기존의 고객 아이디와 새롭게 만들 고객 아이디를 짝지어 딕셔너리로 만들고 이를 기존의 고객 아이디에 mapping하여 진행을 할 것입니다.

In [6]:
# 판다스의 unique() 기능을 이용하여 유저 아이디를 user_id라는 변수에 저장합니다.

user_id = user['user_id'].unique()

user_id

array(['--PYPMX8QWg0ioT5zfORmU-S5Lln0lot',
       '-16-xXbeDcvkZJtTpRwMi57Yo2ZQpORv',
       '-1de9sT-MLwVVvnC0ncCLnqEqpSi3XSN', ...,
       'zz-aNy7UWfvyrZxO4Fs4K5ewmqZVaMOs',
       'zznj-LHhddVvuzZmbZpw6MSylLO64982',
       'zzxBQ7i7mttX0cv1GqFuuMstg7keEkdV'], dtype=object)

In [7]:
#새로운 user_id는 연속된 자연수들로 지정합니다. range()를 사용하여 user_id의 개수만큼의 연속된 정수를 만듭니다.

n_user_id = range(user['user_id'].size)

n_user_id

range(0, 10000)

In [8]:
#python 내장 함수인 zip()을 이용하여 기존의 id와 새로운 id를 짝지어 묶습니다.
id_zip = zip(user_id, n_user_id)

#id_zip을 출력하면 아래 결과와 같이 zip object이 출력됩니다.
print(id_zip)

In [9]:
#zip()의 결과를 구체적으로 보기 위하여 list로 변환하여 print하겠습니다. 상위 5개만 출력합니다.
list(zip(user_id, n_user_id))[:5]


[('--PYPMX8QWg0ioT5zfORmU-S5Lln0lot', 0),
 ('-16-xXbeDcvkZJtTpRwMi57Yo2ZQpORv', 1),
 ('-1de9sT-MLwVVvnC0ncCLnqEqpSi3XSN', 2),
 ('-3A3L2jnM55B_Q1bRXMjZ6sPnINIj-Y1', 3),
 ('-3bhcSgPOIdQAPkPNcchxvECGqGQQ78k', 4)]

In [10]:
#새로운 id와 기존의 id가 대응된 딕셔너리 타입 변수를 생성합니다.
id_dict = dict(id_zip)

id_dict

{'--PYPMX8QWg0ioT5zfORmU-S5Lln0lot': 0,
 '-16-xXbeDcvkZJtTpRwMi57Yo2ZQpORv': 1,
 '-1de9sT-MLwVVvnC0ncCLnqEqpSi3XSN': 2,
 '-3A3L2jnM55B_Q1bRXMjZ6sPnINIj-Y1': 3,
 '-3bhcSgPOIdQAPkPNcchxvECGqGQQ78k': 4,
 '-3fmY1WsLkYJwN_8lZQMmxZd6zJTAcT1': 5,
 '-3q-oynqxFEgSHUwX802hpmi1louyQNv': 6,
 '-428TMckUlhn6ptxN7gR2FGaSyXjSnaD': 7,
 '-4O8WnD8dT6nWho-4KbIm6TvnK4BmjX_': 8,
 '-4ltLPS55n6J2wSUCLxEZwxYdeW37cK5': 9,
 '-5BA0EwkyhGLCC8FxzvvDgyrZWYJM33I': 10,
 '-5Cwn2Fcx9j16QSM2-SLiaLMm0sS4E2I': 11,
 '-5o3lkvJctT3uURb5JWPVxe1VjqhyzAi': 12,
 '-622WUNWBtjX5VGKx8UnOtn2NVHD_NaB': 13,
 '-62U2A3KHjNZ2XXmOgQTSWEfPg1RRWWy': 14,
 '-63J8veARgGL3ulnRKblm4xhhwkvjKzG': 15,
 '-6UZWGgl3AAI7Df2sVWLX6oT6zP43zo0': 16,
 '-6jxyh56lSivkbLm3WNGRCmdyrdsBmNW': 17,
 '-71z4lG_D-eKnOmDCJlUaNvVcwd808yw': 18,
 '-75tFsDSoUwapUvwCUHTZiGTGkaSDleQ': 19,
 '-7SKUZkBmbG2ZMvJ0E0jmMDcd8PgmARb': 20,
 '-7uBbvfy4gff6mHV9XotjVO2YlCY2r8v': 21,
 '-8htVW7UIA8qRupSdCx-6PzIXLI_vk2p': 22,
 '-9qbSavSdufdw9JwmiWX1_URT2E2QxFZ': 23,
 '-Ae6T8G5uAZldwUEOTMR-KzG

이제 위에서 구한 id_dict를 기존의 고객 아이디에 mapping을 하여 새로운 고객 아이디를 만들겠습니다.

기존에 map() 메서드를 이용하여 함수를 컬럼에 적용시키는 것을 해보셨을 겁니다. map() 메서드는 함수 말고도 딕셔너리 타입 변수를 인자로 받아서 컬럼에 적용시킬 수 있습니다. 

딕셔너리 타입 변수를 컬럽에 mapping하게 되면 딕셔너리에서 컬럼의 값을 key로 갖는 value를 반환합니다.
 

Hint) mapping 예시

```
data['column'].map(dict or def)
```

In [11]:
# data_logs, order, user에 id_idct를 mapping하여 각 데이터 프레임에 새로운 고객 id컬럼인 n_user_id를 만듭니다.

data_logs["n_user_id"] = data_logs['user_id'].map(id_dict)
order["n_user_id"] = order['user_id'].map(id_dict)
user["n_user_id"] = user['user_id'].map(id_dict)

# 결과 확인을 위해 data_logs에서 user_id, n_user_id 컬럼의 상위 5 rows만 출력해주세요.

data_logs[['user_id', 'n_user_id']].head()


user_id  n_user_id
0  K1d8_t3-QIskaSkrx32oAFu856D8JmLo       3314
1  lwFZ77v_ygk0uU40t1ud3l30EZ6sE2R3       7844
2  mR-bO6hC9g-m8ERXMRQZaRwJFvzNNdd8       7920
3  K1d8_t3-QIskaSkrx32oAFu856D8JmLo       3314
4  Yjny5AchUWLiuv4kdeq50COF-S8OFXPd       5608

 **2. 주문 데이터, 로그 데이터를 concat해주세요**
 
 주문기록은 user_event_log에 기록되지 않습니다.

In [12]:
#주문 데이터의 첫번째 row를 출력합니다.
order.head(1)
# order.iloc[0]

timestamp                           user_id  goods_id  \
0 2018-06-11 00:00:43.032  bvu0aLTqiFDoU-963xnr5nzQWTNLUMjx      1414   

   shop_id  price  n_user_id  
0       38  45000       6241

In [13]:
# 위의 user_id에 해당하는 고객의 log기록을 가져와주세요. 결과는 아래와 같습니다. 
data_logs.loc[data_logs["user_id"] == "bvu0aLTqiFDoU-963xnr5nzQWTNLUMjx"]

# lambda를 이용한 방법입니다.
#data_logs.loc[lambda x : x["user_id"] == 'bvu0aLTqiFDoU-963xnr5nzQWTNLUMjx']

timestamp                           user_id  \
878  2018-06-11 00:06:45.357  bvu0aLTqiFDoU-963xnr5nzQWTNLUMjx   
901  2018-06-11 00:06:54.034  bvu0aLTqiFDoU-963xnr5nzQWTNLUMjx   
1062 2018-06-11 00:08:00.579  bvu0aLTqiFDoU-963xnr5nzQWTNLUMjx   
1259 2018-06-11 00:09:38.881  bvu0aLTqiFDoU-963xnr5nzQWTNLUMjx   
1439 2018-06-11 00:11:04.446  bvu0aLTqiFDoU-963xnr5nzQWTNLUMjx   
1473 2018-06-11 00:11:20.354  bvu0aLTqiFDoU-963xnr5nzQWTNLUMjx   
1526 2018-06-11 00:11:48.284  bvu0aLTqiFDoU-963xnr5nzQWTNLUMjx   
2423 2018-06-11 00:18:21.906  bvu0aLTqiFDoU-963xnr5nzQWTNLUMjx   
2529 2018-06-11 00:19:01.928  bvu0aLTqiFDoU-963xnr5nzQWTNLUMjx   
2758 2018-06-11 00:20:30.432  bvu0aLTqiFDoU-963xnr5nzQWTNLUMjx   
4502 2018-06-11 00:32:29.738  bvu0aLTqiFDoU-963xnr5nzQWTNLUMjx   
5156 2018-06-11 00:37:22.757  bvu0aLTqiFDoU-963xnr5nzQWTNLUMjx   

                  event_origin     event_name  event_goods_id  event_shop_id  \
878     goods_search_result/린넨  app_page_view             NaN            NaN   
901   goods_search_result/린넨바지  app_page_view             NaN            NaN   
1062  goods_search_result/린넨바지  enter_browser          2048.0           46.0   
1259  goods_search_result/린넨바지  app_page_view             NaN            NaN   
1439  goods_search_result/린넨바지  enter_browser          3486.0           38.0   
1473  goods_search_result/린넨바지  app_page_view             NaN            NaN   
1526  goods_search_result/린넨바지  enter_browser          4006.0           24.0   
2423  goods_search_result/린넨바지  app_page_view             NaN            NaN   
2529    goods_search_result/린넨  app_page_view             NaN            NaN   
2758            shops_bookmark  app_page_view             NaN            NaN   
4502            shops_bookmark  app_page_view             NaN            NaN   
5156            shops_bookmark  app_page_view             NaN            NaN   

      n_user_id  
878        6241  
901        6241  
1062       6241  
1259       6241  
1439       6241  
1473       6241  
1526       6241  
2423       6241  
2529       6241  
2758       6241  
4502       6241  
5156       6241

In [14]:
# 해당 아이디를 주문 테이블에서 찾아보면 
# order[order['user_id']== "bvu0aLTqiFDoU-963xnr5nzQWTNLUMjx"]

로그 데이터에서 00시 37분에 마지막 log가 기록되어 있는 것을 확인할 수 있습니다. 또한 주문 데이터에서 해당 고객이 00시 43분에 구매를 한 것을 확인 할 수 있습니다. 이 기록을 바탕으로 해당 고객은 쇼핑몰 즐겨찾기 목록에서 특정 쇼핑몰을 클릭하여 들어간다음 43분에 린넨바지를 구매한 것을 유추할 수 있습니다. 

이 기록들은 구매 고객을 tracking할 때 매우 중요하지만 각 각 나누어 기록되어 있습니다. 따라서 해당 로그데이터와 주문데이터를 concat 해야됩니다.

하지만 log기록엔 없는 구매 고객들이 있습니다. 이들은 전날 기록이 넘어 온 것 고객들로 유추됩니다. 이들을 확인한 다음 제외하고 concat 하도록 하겠습니다.

로그 데이터와 주문 데이터에 동시에 기록된 고객수를 확인하는데에는 집합 타입 변수(set)의 교집합 연산(&)을 이용하겠습니다. 

데이터를 집합으로 변환하면 중복된 원소들은 1개만 남고 반환됩니다. unique() 메서드의 결과와 같다고 생각하시면 됩니다.

코드예시는 아래와 같습니다.

```
#집합 타입으로 변환하는 코드입니다.
set(data['column'])

#집합의 원소의 개수를 반환하는 코드입니다.
len(set 타입 변수)

#두 집합의 공통된 원소들의 집합을 반환하는 코드입니다.
(set 타입 변수) & (set 타입 변수)
```


In [15]:
# 주문한 고객의 수를 user_number_order란 변수에 저장해주세요.
user_set = set(order['user_id'])
user_number_order = len(user_set)

# log에 기록된 고객의 수를 user_number_log란 변수에 저장해주세요.
log_set = set(data_logs['user_id'])
user_number_log = len(log_set)

# 중복된 고객의 수를 user_duplicated란 변수에 저장해주세요.
user_duplicated = len(user_set & log_set)

# 결과를 출력합니다.
print('해당 날짜에 구매한 총 고객수 입니다 :',user_number_order, end ='명\n')
print('해당 날짜 log데이터에 기록된 총 고객수입니다 :',user_number_log,  end ='명\n')
print('중복되는 고객수입니다 :',user_duplicated , end ='명\n')

해당 날짜에 구매한 총 고객수 입니다 : 832명
해당 날짜 log데이터에 기록된 총 고객수입니다 : 9909명
중복되는 고객수입니다 : 742명


In [16]:
# set()과 &연산을 사용하여 중복되는 고객의 목록을 user_concat_list라는 변수에 저장합니다.

user_concat_list = user_set & log_set

# user_concat_list를 출력합니다.
user_concat_list

{'-1de9sT-MLwVVvnC0ncCLnqEqpSi3XSN',
 '-4O8WnD8dT6nWho-4KbIm6TvnK4BmjX_',
 '-4ltLPS55n6J2wSUCLxEZwxYdeW37cK5',
 '-K76uxrcXqUDULcH-OKfoWxfWrH7-bYc',
 '-L2Awbp23c9b1o1R_do--BZEtPivAUua',
 '-RkcyYiat4mnNUK48f_0xL1rWJC1e5Sy',
 '-bW8vR_yKhs40MmASelnYoyY3daCIMkW',
 '-eGH0rie-miqE6oRMLTjyCkZhUWJh_QM',
 '-mZEEW22aqNU6B794YSve6VB0UOFGCwA',
 '-mfvXk0bpG9FF9BxhtsPO2oIPbLzpPtM',
 '04iVDKAyu75QU3KlclcU2sFBtIkjK4Y_',
 '0GuqiwNd8_1TBrw8ig0LzqY6jO61lP9C',
 '0HxrF9vnFhT-1HREcMRGTfgKb0gWFzxo',
 '0NunOZYHrDHIQBSTzia2t2ujljWeFIdU',
 '0RsDTwKEYNt8tlWoNk1kJi8H7yFpK8ll',
 '0Ui4C1Al9g4w_RWzUaNMqxljuQnWqrQ-',
 '0h770WXoLOIoR35Q4fICQHfn77GJK0vq',
 '0iGJ6o5tisw1QIvxL8EvpPqkCfRzqJpS',
 '0kQZr5aGOvPtUXhy8CdCw6K_-ZN0JQR6',
 '0ssxm5cQEJgNRBRIH4qAsqRc0NnsP_oN',
 '0wh_rlOMGI9bIUwE3opajvEY1rx2CdRA',
 '11NVzbZdGxYQE5_-gctsEfldK6wdbtXK',
 '17i5A1oJK4pgHuLplyCYQtaavxVS-sxs',
 '19aRDkVkz0c0jRjkuPT7el0ExIB6CTIO',
 '19vOH1W4-cFanhVEGQd_eJeo6YofJZiP',
 '1M4ut3jN5QclQfsst-YzP_m9BMogaFwx',
 '1SZp7sMFWAfcLfGOZOWKLyDgfaIw2jT7',
 

중복되는 고객의 목록을 구하였으므로 이제는 주문데이터와 로그데이터를 concat하기 위해 column명을 동일하게 맞추겠습니다.

우선 order, data_logs의 column을 확인하겠습니다.

In [17]:
print(order.columns)
print(data_logs.columns)

Index(['timestamp', 'user_id', 'goods_id', 'shop_id', 'price', 'n_user_id'], dtype='object')
Index(['timestamp', 'user_id', 'event_origin', 'event_name', 'event_goods_id',
       'event_shop_id', 'n_user_id'],
      dtype='object')


data_logs의 columns을 기준으로 병합할 것입니다. 따라서 order의 columns을 data_logs에 맞춰 변형합니다.

In [18]:
# order 원본을 변형하지 않기 위해 order을 copy하여 사용합니다. 
df = order.copy()
df.head()

timestamp                           user_id  goods_id  \
0 2018-06-11 00:00:43.032  bvu0aLTqiFDoU-963xnr5nzQWTNLUMjx      1414   
1 2018-06-11 00:02:33.763  smDmRnykg61KajpxXKzQ0oNkrh2nuSBj      1351   
2 2018-06-11 00:04:06.364  EyGjKYtSqZgqJ1ddKCtH5XwGirTyOH2P       646   
3 2018-06-11 00:04:17.258  KQBGi33Zxh5Dgu0WEkOkjN0YqTT_wxC3      5901   
4 2018-06-11 00:05:26.010  lq1Je3voA3a0MouSFba3629lKCvweI24      5572   

   shop_id  price  n_user_id  
0       38  45000       6241  
1       12   9500       8899  
2       14  22000       2527  
3       46  29800       3387  
4       89  29000       7832

In [19]:
# 겹치는 유저만으로 data를 indexing합니다. user_concat_list를 이용합니다. 
# isin을 이용한 indexing은 뒤의 문제에서도 많이 나오기 때문에 유심히 봐두시면 유용합니다.

print(df.shape)
df = df[df['user_id'].isin(user_concat_list)]
print(df.shape)

#  event_origin 컬럼에는 shop_id 컬럼을 저장합니다.
df['event_origin'] = df['shop_id']

#  event_name 컬럼에는 'purchase'를 저장하여 추가한다
df['event_name'] = 'purchase'

# event_goods_id 컬럼에는 good_id 컬럼를 저장합니다.
df['event_goods_id'] = df['goods_id']

# 사용할 columns를 설정합니다.
df = df[['timestamp', 'n_user_id', 'user_id','event_origin',
         'event_name', 'event_goods_id', 'price']]

df.head()


(867, 6)
(772, 6)


timestamp  n_user_id                           user_id  \
0 2018-06-11 00:00:43.032       6241  bvu0aLTqiFDoU-963xnr5nzQWTNLUMjx   
1 2018-06-11 00:02:33.763       8899  smDmRnykg61KajpxXKzQ0oNkrh2nuSBj   
4 2018-06-11 00:05:26.010       7832  lq1Je3voA3a0MouSFba3629lKCvweI24   
5 2018-06-11 00:05:35.182       2745  GM0-EsJPHjkpteIpAQIwaCdUjU81lhW1   
6 2018-06-11 00:06:14.314       7800  lgvWxrv7r5RGklXSJqM2x6NUBZ5H-RQZ   

   event_origin event_name  event_goods_id  price  
0            38   purchase            1414  45000  
1            12   purchase            1351   9500  
4            89   purchase            5572  29000  
5            22   purchase              55  11200  
6            22   purchase            2451  19800

In [20]:
#  다시 data_logs 데이타 확인 - 비교하기 위해
data_logs.head(5)

timestamp                           user_id  \
0 2018-06-11 00:00:00.213  K1d8_t3-QIskaSkrx32oAFu856D8JmLo   
1 2018-06-11 00:00:00.810  lwFZ77v_ygk0uU40t1ud3l30EZ6sE2R3   
2 2018-06-11 00:00:00.956  mR-bO6hC9g-m8ERXMRQZaRwJFvzNNdd8   
3 2018-06-11 00:00:01.084  K1d8_t3-QIskaSkrx32oAFu856D8JmLo   
4 2018-06-11 00:00:01.561  Yjny5AchUWLiuv4kdeq50COF-S8OFXPd   

             event_origin     event_name  event_goods_id  event_shop_id  \
0           shops_ranking  app_page_view             NaN            NaN   
1          shops_bookmark  app_page_view             NaN            NaN   
2  goods_search_result/로브  app_page_view             NaN            NaN   
3          shops_bookmark  app_page_view             NaN            NaN   
4          shops_bookmark  app_page_view             NaN            NaN   

   n_user_id  
0       3314  
1       7844  
2       7920  
3       3314  
4       5608

In [21]:

# data_logs, df를 concat 하고 data_logs_concated에 저장해주세요.

data_logs_concated = pd.concat([data_logs, df], sort=False)

data_logs_concated.shape


(106587, 8)

 data_logs_concated에 구매기록여부 컬럼인 purchase 컬럼을 만들어주세요.
 
 price컬럼을 이용해주세요. 
 
 출력 결과는 아래와 같습니다.
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>event_goods_id</th>
      <th>event_name</th>
      <th>event_origin</th>
      <th>event_shop_id</th>
      <th>n_user_id</th>
      <th>price</th>
      <th>timestamp</th>
      <th>user_id</th>
      <th>purchase</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>NaN</td>
      <td>app_page_view</td>
      <td>shops_ranking</td>
      <td>NaN</td>
      <td>3314</td>
      <td>NaN</td>
      <td>2018-06-11 00:00:00.213</td>
      <td>K1d8_t3-QIskaSkrx32oAFu856D8JmLo</td>
      <td>False</td>
    </tr>
    <tr>
      <th>1</th>
      <td>NaN</td>
      <td>app_page_view</td>
      <td>shops_bookmark</td>
      <td>NaN</td>
      <td>7844</td>
      <td>NaN</td>
      <td>2018-06-11 00:00:00.810</td>
      <td>lwFZ77v_ygk0uU40t1ud3l30EZ6sE2R3</td>
      <td>False</td>
    </tr>
    <tr>
      <th>2</th>
      <td>NaN</td>
      <td>app_page_view</td>
      <td>goods_search_result/로브</td>
      <td>NaN</td>
      <td>7920</td>
      <td>NaN</td>
      <td>2018-06-11 00:00:00.956</td>
      <td>mR-bO6hC9g-m8ERXMRQZaRwJFvzNNdd8</td>
      <td>False</td>
    </tr>
    <tr>
      <th>3</th>
      <td>NaN</td>
      <td>app_page_view</td>
      <td>shops_bookmark</td>
      <td>NaN</td>
      <td>3314</td>
      <td>NaN</td>
      <td>2018-06-11 00:00:01.084</td>
      <td>K1d8_t3-QIskaSkrx32oAFu856D8JmLo</td>
      <td>False</td>
    </tr>
    <tr>
      <th>4</th>
      <td>NaN</td>
      <td>app_page_view</td>
      <td>shops_bookmark</td>
      <td>NaN</td>
      <td>5608</td>
      <td>NaN</td>
      <td>2018-06-11 00:00:01.561</td>
      <td>Yjny5AchUWLiuv4kdeq50COF-S8OFXPd</td>
      <td>False</td>
    </tr>
  </tbody>
</table>


----

In [22]:
data_logs_concated['purchase'] = data_logs_concated["price"].notnull()

print(data_logs_concated.shape)
data_logs_concated.head()


(106587, 9)


timestamp                           user_id  \
0 2018-06-11 00:00:00.213  K1d8_t3-QIskaSkrx32oAFu856D8JmLo   
1 2018-06-11 00:00:00.810  lwFZ77v_ygk0uU40t1ud3l30EZ6sE2R3   
2 2018-06-11 00:00:00.956  mR-bO6hC9g-m8ERXMRQZaRwJFvzNNdd8   
3 2018-06-11 00:00:01.084  K1d8_t3-QIskaSkrx32oAFu856D8JmLo   
4 2018-06-11 00:00:01.561  Yjny5AchUWLiuv4kdeq50COF-S8OFXPd   

             event_origin     event_name  event_goods_id  event_shop_id  \
0           shops_ranking  app_page_view             NaN            NaN   
1          shops_bookmark  app_page_view             NaN            NaN   
2  goods_search_result/로브  app_page_view             NaN            NaN   
3          shops_bookmark  app_page_view             NaN            NaN   
4          shops_bookmark  app_page_view             NaN            NaN   

   n_user_id  price  purchase  
0       3314    NaN     False  
1       7844    NaN     False  
2       7920    NaN     False  
3       3314    NaN     False  
4       5608    NaN     False

----

데이터를 파악하기 쉽게 하기 위해 user_id, timestamp 컬럼을 기준으로 data_logs_concated를 정렬해주세요.

sort_values()를 사용해주시고 reset_index()를 통해 index를 정리해주세요.

출력 결과는 아래와 같습니다.

[참고] [reset_index 함수](https://datascienceschool.net/view-notebook/a49bde24674a46699639c1fa9bb7e213/)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>event_goods_id</th>
      <th>event_name</th>
      <th>event_origin</th>
      <th>event_shop_id</th>
      <th>n_user_id</th>
      <th>price</th>
      <th>timestamp</th>
      <th>user_id</th>
      <th>purchase</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>NaN</td>
      <td>app_page_view</td>
      <td>shops_bookmark</td>
      <td>NaN</td>
      <td>0</td>
      <td>NaN</td>
      <td>2018-06-11 15:57:10.615</td>
      <td>--PYPMX8QWg0ioT5zfORmU-S5Lln0lot</td>
      <td>False</td>
    </tr>
    <tr>
      <th>1</th>
      <td>NaN</td>
      <td>app_page_view</td>
      <td>shops_bookmark</td>
      <td>NaN</td>
      <td>0</td>
      <td>NaN</td>
      <td>2018-06-11 15:59:05.505</td>
      <td>--PYPMX8QWg0ioT5zfORmU-S5Lln0lot</td>
      <td>False</td>
    </tr>
    <tr>
      <th>2</th>
      <td>NaN</td>
      <td>app_page_view</td>
      <td>my_goods</td>
      <td>NaN</td>
      <td>1</td>
      <td>NaN</td>
      <td>2018-06-11 00:55:37.309</td>
      <td>-16-xXbeDcvkZJtTpRwMi57Yo2ZQpORv</td>
      <td>False</td>
    </tr>
    <tr>
      <th>3</th>
      <td>2506.0</td>
      <td>enter_browser</td>
      <td>my_goods</td>
      <td>40.0</td>
      <td>1</td>
      <td>NaN</td>
      <td>2018-06-11 00:55:44.430</td>
      <td>-16-xXbeDcvkZJtTpRwMi57Yo2ZQpORv</td>
      <td>False</td>
    </tr>
    <tr>
      <th>4</th>
      <td>NaN</td>
      <td>app_page_view</td>
      <td>my_goods</td>
      <td>NaN</td>
      <td>1</td>
      <td>NaN</td>
      <td>2018-06-11 01:00:33.295</td>
      <td>-16-xXbeDcvkZJtTpRwMi57Yo2ZQpORv</td>
      <td>False</td>
    </tr>
  </tbody>
</table>


In [23]:
# data_logs_concated.sort_values(by = ['user_id', 'timestamp'])

In [24]:

data_logs_concated = data_logs_concated.sort_values(by = ['user_id', 'timestamp']).reset_index(drop=True)

print(data_logs_concated.shape)
data_logs_concated.head()


(106587, 9)


timestamp                           user_id    event_origin  \
0 2018-06-11 15:57:10.615  --PYPMX8QWg0ioT5zfORmU-S5Lln0lot  shops_bookmark   
1 2018-06-11 15:59:05.505  --PYPMX8QWg0ioT5zfORmU-S5Lln0lot  shops_bookmark   
2 2018-06-11 00:55:37.309  -16-xXbeDcvkZJtTpRwMi57Yo2ZQpORv        my_goods   
3 2018-06-11 00:55:44.430  -16-xXbeDcvkZJtTpRwMi57Yo2ZQpORv        my_goods   
4 2018-06-11 01:00:33.295  -16-xXbeDcvkZJtTpRwMi57Yo2ZQpORv        my_goods   

      event_name  event_goods_id  event_shop_id  n_user_id  price  purchase  
0  app_page_view             NaN            NaN          0    NaN     False  
1  app_page_view             NaN            NaN          0    NaN     False  
2  app_page_view             NaN            NaN          1    NaN     False  
3  enter_browser          2506.0           40.0          1    NaN     False  
4  app_page_view             NaN            NaN          1    NaN     False

**3. page duration 을 구해주세요.**

page duration은 사용자가 앱의 한 page당 체류하는 시간입니다. 이는 동일한 사용자에 대한 연속한 로그들 사이의 시간 간격을 뜻합니다.

이를 구하기 위하여, 로그별로 연속된 다음 로그의 timestamp를 저장하는timestamp_after라는 컬럼을 만들고 timestamp와의 차이를 계산할 것입니다.

현재 제공된 데이터에서는 고객이 앱을 종료하는 기록이 없기 때문에 마지막 log의 page_duration은 0이라고 가정을 합니다.

------

timestamp_after 컬럼을 만들어줍니다.

고객별로 groupby()한 뒤 shift(-1)을 적용한 결과를 timestamp_after에 저장합니다. 

groupby()를 하고 shift()를 하면 고객의 당일 마지막 로그의 timestamp_after은 NaT(Not a Time)값을 갖게 됩니다.


In [25]:
# timestamp_after 컬럼을 다음과 같이 만듭니다.

data_logs_concated['timestamp_after'] = data_logs_concated.groupby(['n_user_id'])['timestamp'].shift(-1)
print(data_logs_concated.shape)
data_logs_concated.head()

(106587, 10)


timestamp                           user_id    event_origin  \
0 2018-06-11 15:57:10.615  --PYPMX8QWg0ioT5zfORmU-S5Lln0lot  shops_bookmark   
1 2018-06-11 15:59:05.505  --PYPMX8QWg0ioT5zfORmU-S5Lln0lot  shops_bookmark   
2 2018-06-11 00:55:37.309  -16-xXbeDcvkZJtTpRwMi57Yo2ZQpORv        my_goods   
3 2018-06-11 00:55:44.430  -16-xXbeDcvkZJtTpRwMi57Yo2ZQpORv        my_goods   
4 2018-06-11 01:00:33.295  -16-xXbeDcvkZJtTpRwMi57Yo2ZQpORv        my_goods   

      event_name  event_goods_id  event_shop_id  n_user_id  price  purchase  \
0  app_page_view             NaN            NaN          0    NaN     False   
1  app_page_view             NaN            NaN          0    NaN     False   
2  app_page_view             NaN            NaN          1    NaN     False   
3  enter_browser          2506.0           40.0          1    NaN     False   
4  app_page_view             NaN            NaN          1    NaN     False   

          timestamp_after  
0 2018-06-11 15:59:05.505  
1                     NaT  
2 2018-06-11 00:55:44.430  
3 2018-06-11 01:00:33.295  
4 2018-06-11 01:11:03.608

이제 timestamp_after과 timestamp의 차이를 계산하여 page_duration을 구한후 . NaT 값은 연산시 NaN값이 됩니다. 이후에 NaN값을 0으로 채워주시면 됩니다.

출력 결과는 아래와 같습니다.

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>timestamp</th>
      <th>timestamp_after</th>
      <th>page_duration</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>2018-06-11 15:57:10.615</td>
      <td>2018-06-11 15:59:05.505</td>
      <td>00:01:54.890000</td>
    </tr>
    <tr>
      <th>1</th>
      <td>2018-06-11 15:59:05.505</td>
      <td>NaT</td>
      <td>00:00:00</td>
    </tr>
    <tr>
      <th>2</th>
      <td>2018-06-11 00:55:37.309</td>
      <td>2018-06-11 00:55:44.430</td>
      <td>00:00:07.121000</td>
    </tr>
    <tr>
      <th>3</th>
      <td>2018-06-11 00:55:44.430</td>
      <td>2018-06-11 01:00:33.295</td>
      <td>00:04:48.865000</td>
    </tr>
    <tr>
      <th>4</th>
      <td>2018-06-11 01:00:33.295</td>
      <td>2018-06-11 01:11:03.608</td>
      <td>00:10:30.313000</td>
    </tr>
  </tbody>
</table>



In [26]:
# timestamp 형식끼리 연산을 해야 하기에
data_logs_concated.info()  # 먼저 자료형 확인
data_logs_concated['timestamp'] = pd.to_datetime(data_logs_concated['timestamp'])
data_logs_concated.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106587 entries, 0 to 106586
Data columns (total 10 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   timestamp        106587 non-null  datetime64[ns]
 1   user_id          106587 non-null  object        
 2   event_origin     106587 non-null  object        
 3   event_name       106587 non-null  object        
 4   event_goods_id   19293 non-null   float64       
 5   event_shop_id    21644 non-null   float64       
 6   n_user_id        106587 non-null  int64         
 7   price            772 non-null     float64       
 8   purchase         106587 non-null  bool          
 9   timestamp_after  96678 non-null   datetime64[ns]
dtypes: bool(1), datetime64[ns](2), float64(3), int64(1), object(3)
memory usage: 7.4+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106587 entries, 0 to 106586
Data columns (total 10 columns):
 #   Column           Non-Null Count   D

In [45]:
# Passing integers to fillna for timedelta64[ns] dtype is no longer supported.  To obtain the old behavior, pass `pd.Timedelta(seconds=n)` instead.

data_logs_concated['page_duration'] = (data_logs_concated['timestamp_after'] - data_logs_concated['timestamp']).fillna(pd.Timedelta(0))
print(data_logs_concated.shape)
data_logs_concated[['timestamp','timestamp_after','page_duration']].head()


(106587, 11)


timestamp         timestamp_after   page_duration
0 2018-06-11 15:57:10.615 2018-06-11 15:59:05.505 00:01:54.890000
1 2018-06-11 15:59:05.505                     NaT        00:00:00
2 2018-06-11 00:55:37.309 2018-06-11 00:55:44.430 00:00:07.121000
3 2018-06-11 00:55:44.430 2018-06-11 01:00:33.295 00:04:48.865000
4 2018-06-11 01:00:33.295 2018-06-11 01:11:03.608 00:10:30.313000

분석의 편의를 위하여 map()과 total_seconds() 메서드를 이용하여 page_duration을 초로 환산해주세요.

마지막으로, 이 후의 계산을 위해 astype() 메서드를 이용하여 page_duration 컬럼을 float으로 변환해주세요.  



출력 결과는 아래와 같습니다.

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>n_user_id</th>
      <th>timestamp</th>
      <th>timestamp_after</th>
      <th>page_duration</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>0</td>
      <td>2018-06-11 15:57:10.615</td>
      <td>2018-06-11 15:59:05.505</td>
      <td>114.890</td>
    </tr>
    <tr>
      <th>1</th>
      <td>0</td>
      <td>2018-06-11 15:59:05.505</td>
      <td>NaT</td>
      <td>0.000</td>
    </tr>
    <tr>
      <th>2</th>
      <td>1</td>
      <td>2018-06-11 00:55:37.309</td>
      <td>2018-06-11 00:55:44.430</td>
      <td>7.121</td>
    </tr>
    <tr>
      <th>3</th>
      <td>1</td>
      <td>2018-06-11 00:55:44.430</td>
      <td>2018-06-11 01:00:33.295</td>
      <td>288.865</td>
    </tr>
    <tr>
      <th>4</th>
      <td>1</td>
      <td>2018-06-11 01:00:33.295</td>
      <td>2018-06-11 01:11:03.608</td>
      <td>630.313</td>
    </tr>
  </tbody>
</table>



In [ ]:
data_logs_concated['page_duration'] = data_logs_concated['page_duration'].map(lambda x: x.total_seconds()).astype('float') 

data_logs_concated[['n_user_id','timestamp', 'timestamp_after', 'page_duration']].head()

**4. session을 구해주세요.**

사용자가 앱을 실행하는 단위를 세션(session)이라고 정의합니다. 세션은 사용자가 앱을 실행한 후부터 그 실행을 마칠 때까지의 일련의 과정을 포함합니다.

session을 구하기 위해서 고객이 session을 종료하고 앱을 나갔는지 여부(boolean)인 is_out이라는 컬럼을 만듭니다.

is_out의 조건은 아래와 같이 2가지가 있습니다.

    조건 1) page_duration이 0 이면 고객의 당일 마지막 log이기 때문에 고객이 session을 종료하고 앱을 나갔다고 가정합니다. (page_duration 설명 부분에서 가정하였습니다.)
    
    조건 2) page_duration이 30분이상이면 고객이 한 session을 종료한 것이라고 가정하겠습니다.
   

따라서 위의 조건중 적어도 1개를 만족시키키면 True되도록 is_out 컬럼을 생성해주세요. 





출력 결과는 아래와 같습니다.
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>n_user_id</th>
      <th>timestamp</th>
      <th>timestamp_after</th>
      <th>page_duration</th>
      <th>is_out</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>0</td>
      <td>2018-06-11 15:57:10.615</td>
      <td>2018-06-11 15:59:05.505</td>
      <td>114.890</td>
      <td>False</td>
    </tr>
    <tr>
      <th>1</th>
      <td>0</td>
      <td>2018-06-11 15:59:05.505</td>
      <td>NaT</td>
      <td>0.000</td>
      <td>True</td>
    </tr>
    <tr>
      <th>2</th>
      <td>1</td>
      <td>2018-06-11 00:55:37.309</td>
      <td>2018-06-11 00:55:44.430</td>
      <td>7.121</td>
      <td>False</td>
    </tr>
    <tr>
      <th>3</th>
      <td>1</td>
      <td>2018-06-11 00:55:44.430</td>
      <td>2018-06-11 01:00:33.295</td>
      <td>288.865</td>
      <td>False</td>
    </tr>
    <tr>
      <th>4</th>
      <td>1</td>
      <td>2018-06-11 01:00:33.295</td>
      <td>2018-06-11 01:11:03.608</td>
      <td>630.313</td>
      <td>False</td>
    </tr>
  </tbody>
</table>




In [ ]:
# 30분을 초로 환산해주셔야 합니다.

data_logs_concated["is_out"] = (data_logs_concated["page_duration"] > 30*60) | (data_logs_concated["page_duration"] == 0 )

data_logs_concated[['n_user_id','timestamp', 'timestamp_after', 'page_duration', 'is_out']].head(10)
# data_logs.loc[data_logs['page_duration'] ==0, "is_out"] = True

page_duration이 1800(30분)이상 또는 0인 log들은 session의 마지막 log입니다. 예를 들어, page_duration이 36000인 log가 있으면 이는 10시간 뒤에 다시 app에 접속한다는 것을 의미하는 것이지 실제 10시간 동안 앱을 사용했다는 의미가 아닙니다. 따라서 이 사람에 대한 해당 log는 해당 session의 마지막 log가 되고 다음 log는 10시간 뒤인 다음 session의 첫 log가 됩니다.

이제 is_out 컬럼을 만들었으니 is_out이 True인 log들, 즉 session의 마지막 log들의 page_duration은 0이 되도록 변환해주어야 합니다.

is_out 컬럼이 True인 page_duration을 0으로 지정해주세요.

In [ ]:
data_logs_concated.loc[data_logs_concated['is_out'] == True, 'page_duration']
# data_logs_concated['is_out'] == True 인 경우 page_duration 가 0인 경우가 거의 대부분이지만 그래도 0으로 지정 (다음)

In [ ]:
data_logs_concated.loc[data_logs_concated['is_out'] == True, 'page_duration'] = 0

data_logs_concated[['n_user_id','timestamp', 'timestamp_after', 'page_duration', 'is_out']].head(20)

----

이제 session을 구하여 번호를 부여할 것입니다.

번호는 session별 고유 번호(session_idx_unique)와 고객별 0부터 시작하는 daily session 번호(session_idx_daily)를 부여합니다.

In [ ]:
# 아래 코드에 대해 단계별로 데이타를 확인하기 위해
data_logs_concated['is_out-cumsum()'] = data_logs_concated['is_out'].cumsum() # 누적합
data_logs_concated["is_out-cumsum()-shift(1)"] = data_logs_concated['is_out-cumsum()'].shift(1)
data_logs_concated["is_out-cumsum()-shift(1)-fillna(0)"] = data_logs_concated["is_out-cumsum()-shift(1)"].fillna(0)
data_logs_concated["is_out-cumsum()-shift(1)-fillna(0)-astype(int)"] = data_logs_concated["is_out-cumsum()-shift(1)-fillna(0)"].astype(int)


In [ ]:
data_logs_concated['session_idx_unique'] = (data_logs_concated['is_out']
                                 .cumsum() # 컬럼의 누적 합계를 반환합니다.
                                 .shift(1) # 결과를 확인하면 왜 shift 하는지 알게된다
                                 .fillna(0)
                                 .astype(int)
                                )

data_logs_concated[['n_user_id','timestamp', 'timestamp_after', 'page_duration', 'is_out','is_out','is_out-cumsum()','is_out-cumsum()-shift(1)', 'is_out-cumsum()-shift(1)-fillna(0)','is_out-cumsum()-shift(1)-fillna(0)-astype(int)','session_idx_unique']].head(30)

이번엔 daily session 번호를 부여하겠습니다. 위의 고유 번화와는 다르게 groupby() 메서드를 사용해야 해서 shift() 메서드를 먼저 적용해야 합니다. 

In [ ]:
# data_logs_concated.groupby('n_user_id')['is_out'].shift(1)

# 아래 코드에 대해 단계별로 데이타를 확인하기 위해
data_logs_concated['is_out-shift(1)'] = data_logs_concated.groupby('n_user_id')['is_out'].shift(1)
data_logs_concated["is_out-shift(1)-fillna(0)"] = data_logs_concated.groupby('n_user_id')['is_out'].shift(1).fillna(0)
data_logs_concated["is_out-shift(1)-fillna(0)-cumsum()"] = data_logs_concated.groupby('n_user_id')['is_out'].shift(1).fillna(0).cumsum()
data_logs_concated["is_out-shift(1)-fillna(0)-cumsum()-astype(int)"] = data_logs_concated.groupby('n_user_id')['is_out'].shift(1).fillna(0).cumsum().astype(int)


In [ ]:
# 결과를 보면 n_user_id가 2인 사람이 접속했다가 다시 접속하는 부분을 확인 할 수 있다
data_logs_concated['session_idx_daily'] = (data_logs_concated.groupby('n_user_id')['is_out']
                                        .shift(1)
                                        .fillna(0)
                                        .cumsum()
                                        .astype(int)
                                    )
print(data_logs_concated.shape)
data_logs_concated[['n_user_id','timestamp', 'timestamp_after', 'page_duration', 'is_out','is_out','is_out-shift(1)','is_out-shift(1)-fillna(0)', 'is_out-shift(1)-fillna(0)-cumsum()','is_out-shift(1)-fillna(0)-cumsum()-astype(int)','session_idx_daily']].head(30)


이번엔 session을 이용한 분석을 해보겠습니다.

session별 log 수(접속별 활동 개수), user별 session별 평균 log수 (고객별 접속당 평균 활동수)을 구하겠습니다.

session별 log 수는 코드는 아래와 같습니다.

이를 활용하여 4.2)user별 session당 평균 log수를 구해주세요.

**4.1) session별 log 수**

In [ ]:
session_log_count = (data_logs_concated
                 .groupby(['n_user_id', 'session_idx_daily']) 
                 .size()  # 그룹별 속한 row 수(log 수)를 반환합니다.
                 .reset_index()  # index를 초기하여 groupby object가 아닌 일반적인 data frame형태로 바꿉니다.
                 .rename(columns = { 0 : "log_count"} )   # 컬럼이름을 log_count로 변경합니다.
                )


print(session_log_count.shape)
session_log_count.head(20)

**4.2) user별 session당 log수의 평균**

In [ ]:
session_user_log_count = (
                        session_log_count
                        .groupby('n_user_id')['log_count']
                        .mean()
                        .reset_index()
                        .rename(columns = {'log_count' : 'log_count_mean'})
                    )

session_user_log_count.head(20)

**4.3) 가장 많은 session을 갖는(가장 많이 활동한) user의 session수를 구해주세요.**

Hint) groupby(), nunique(), max() 메서드

정답은 ``11`` 입니다.

나는 왜 12가 나올까? 위에 데이타에서 부터 틀렸군

In [ ]:
user_session_cnt = (data_logs_concated
 .groupby(["n_user_id"])['session_idx_daily']
 .nunique()
 .max())
print(user_session_cnt)

**5. 기준별 체류 시간을 구해주세요**

체류시간이란 고객이 앱에서 머문 시간을 뜻합니다.



체류시간이 높다는 것은 사이트 운영의 청신호라고 생각할 수 있습니다. 일단 방문 목적과 랜딩페이지에서 제공되는 컨텐츠가 부합한다는 뜻이며, 웹사이트의 컨텐츠에 흥미를 느낀 방문자들이 계속 머물고 있다는 뜻이기도 합니다. 방문자들이 웹사이트에 오랜 시간 머물게 되면 웹사이트에서 제공하는 다양한 장치들을 접할 기회가 많아지고 전환에 도달할 확률이 더욱 높아지기 때문에 체류시간은 전환에 있어 매우 중요한 요소입니다.
광고를 고객을 통해 app으로의 유입을 성공해다면 그 다음 목표는 방문자를 계속 머물게 하여 전환으로 이어질 수 있도록 하는 것입니다.

해당 수업의 zigzag 데이터는 짧은 시간의 제한된 데이터라 체류시간을 이용하여 분석할 수 있는게 많지 않지만 현업에서는 체류시간을 이용하여 통해 어떤 채널을 이용한 고객 또는 어떤 광고를 통해 유입된 고객이 웹사이트/app에 오래 머물고 제품을 구매하는지에 대한 분석 또는 시간대/요일별 노출전략을 세우는 등 다양한 insight를 얻을 수 있습니다.



두가지 기준으로 체류시간을 구할 것입니다.

    1) user별
    2) 구매 user/ 비구매 유저

5.1) user별 체류시간을 구해주세요.

이는 두 단계로 구분됩니다.
1. session별 체류시간 구하기.
2. session별 체류시간을 바탕으로 user별 체류시간 구하기



먼저 session별 체류시간을 다음과 같이 구합니다. session 번호별 page_duration들의 합을 구하면 됩니다.

In [ ]:
duration_session = (data_logs_concated
                        .groupby(['n_user_id', 'session_idx_daily'])['page_duration']
                         .sum()
                         .reset_index()
                         .rename(columns = {'page_duration' : 'duration'}))

duration_session.head(10)


이제 duration_session을 이용하여 user별 체류시간을 구해주세요.

Hint) groupby(), mean() 메서드

결과는 아래와 같습니다.

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>duration</th>
    </tr>
    <tr>
      <th>n_user_id</th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>114.89000</td>
    </tr>
    <tr>
      <th>1</th>
      <td>1011.54100</td>
    </tr>
    <tr>
      <th>2</th>
      <td>463.03625</td>
    </tr>
    <tr>
      <th>3</th>
      <td>460.53125</td>
    </tr>
    <tr>
      <th>4</th>
      <td>49.13000</td>
    </tr>
  </tbody>
</table>


In [ ]:
duration_user = duration_session.groupby("n_user_id")['duration'].mean()

duration_user.to_frame().head()

잔존 시간이 0인 고객들은 app에 들어와서 아무것도 안하고 나간 고객들입니다.

제외하고 계산할 수도 있습니다.

**5.2) 구매/비구매 session별 평균 체류시간 구하기**

구매 기록이 있는 session은 체류시간이 길 것이라고 예상할 수 있습니다. 이를 확인하기 위하여 구매/비구매 session별 평균 체류시간을 구하겠습니다.

먼저 구매기록이 있는 session list를 만들어 주세요.

session_purchse 변수에 구매기록이 있는 session들을 저장해주시면 됩니다.

Hint) purchase 컬럼, indexing, unique() 메서드

In [ ]:
# data_logs_concated.info()
data_logs_concated.head()

In [ ]:
session_purchase = data_logs_concated.loc[data_logs_concated['purchase']==True, 'session_idx_unique'].unique()

session_purchase

이번에는 data_purchase 변수에는 구매 기록이 있는 session들의 데이터를,

data_npurchase 변수에는 구매 기록이 없는 session들의 데이터를 저장해주세요.

Hint) inin()메서드를 이용한 indexing

In [ ]:
data_purchase = data_logs_concated.loc[data_logs_concated['session_idx_unique'].isin(session_purchase)]
data_npurchase = data_logs_concated.loc[~data_logs_concated['session_idx_unique'].isin(session_purchase)]

----

이제 data_purchase를 이용하여 구매 session의 체류시간을 구해주세요.

Hint) User별 체류시간을 구하는 방법과 매우 유사합니다. session_idx_unique를 이용해주세요.

결과는 아래와 같습니다.

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>n_user_id</th>
      <th>session_idx_unique</th>
      <th>session_duration</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>2</td>
      <td>5</td>
      <td>389.636</td>
    </tr>
    <tr>
      <th>1</th>
      <td>8</td>
      <td>16</td>
      <td>412.025</td>
    </tr>
    <tr>
      <th>2</th>
      <td>8</td>
      <td>17</td>
      <td>1791.231</td>
    </tr>
    <tr>
      <th>3</th>
      <td>9</td>
      <td>20</td>
      <td>1657.393</td>
    </tr>
    <tr>
      <th>4</th>
      <td>47</td>
      <td>92</td>
      <td>1214.052</td>
    </tr>
    <tr>
      <th>5</th>
      <td>49</td>
      <td>96</td>
      <td>2615.611</td>
    </tr>
    <tr>
      <th>6</th>
      <td>65</td>
      <td>130</td>
      <td>3093.858</td>
    </tr>
    <tr>
      <th>7</th>
      <td>86</td>
      <td>176</td>
      <td>1906.721</td>
    </tr>
    <tr>
      <th>8</th>
      <td>97</td>
      <td>202</td>
      <td>3031.770</td>
    </tr>
    <tr>
      <th>9</th>
      <td>117</td>
      <td>242</td>
      <td>1191.110</td>
    </tr>
  </tbody>
</table>


In [ ]:
purchase_session_duration = (
                    data_purchase
                    .groupby(['n_user_id', 'session_idx_unique'])['page_duration']
                    .sum()
                    .reset_index()
                    .rename(columns = {"page_duration" : 'session_duration'})
                )

purchase_session_duration.head(10)

마찬가지로 비구매 session의 체류시간을 구해주세요.

In [ ]:
npurchase_session_duration = (
                    data_npurchase
                    .groupby(['n_user_id', 'session_idx_unique'])['page_duration']
                    .sum()
                    .reset_index()
                    .rename(columns = {"page_duration" : 'session_duration'})
                )

npurchase_session_duration.head(10)

마지막으로 구매 session 잔존 시간 평균과 비구매 session 잔존 시간 평균을 구하여 각 각 purchase_session_mean, npurchase_session_mean 변수에 저장해주세요. 


In [ ]:
purchase_session_mean = purchase_session_duration['session_duration'].mean()
npurchase_session_mean = npurchase_session_duration['session_duration'].mean()

print("구매 session 잔존 시간 평균:",purchase_session_mean)
print("비구매 session 잔존 시간 평균:",npurchase_session_mean)